In [41]:
import os 
import numpy as np
import k3d

In [42]:
def obj_data_to_mesh3d(file_path):
    vertices = []
    faces = []
    with open(file_path, "r") as file:
        for line in file:
            slist = line.split()
            if slist:
                if slist[0] == 'v':
                    vertex = np.array(slist[1:], dtype=float)
                    vertices.append(vertex)
                elif slist[0] == 'f':
                    face = []
                    for k in range(1, len(slist)):
                        face.append([int(s) for s in slist[k].replace('//','/').split('/')])
                    if len(face) > 3: # triangulate the n-polyonal face, n>3
                        faces.extend([[face[0][0]-1, face[k][0]-1, face[k+1][0]-1] for k in range(1, len(face)-1)])
                    else:
                        faces.append([face[j][0]-1 for j in range(len(face))])
                else: pass


    return np.array(vertices), np.array(faces)

In [43]:
def get_colors(color_scheme, spacing, w):
    colors = []
    norm_w = w / 100 #100 # Percent error is a percent, we want decimals.
    for v in norm_w:
        color = None
        for idx in range(len(spacing)):
            if v < spacing[idx]:
                color = color_scheme[idx]
                break
        if color is None:
            color = color_scheme[len(color_scheme) - 1]

        colors.append(color)
    return colors

In [44]:
input_path  = "test_seed=0_factor=1.3_dist=20_bound+10_pred_obj/FA3_8L20221001_0401.obj"
save_path = "screenshot.png"
#color_scheme = [0x00008A, 0xFFD600, 0xFF0000]
color_scheme = [0x00008A, 0x00FF00]

In [45]:
OFFSET = 1e-6

spacing = []
space = (1 - OFFSET) / (len(color_scheme) - 1)
for idx in range(1, len(color_scheme)):
    spacing.append(space * idx + OFFSET)
spacing.insert(0, OFFSET)

input_dir, filename = os.path.split(input_path)

In [46]:
vertices, faces = obj_data_to_mesh3d(os.path.join(input_dir, filename))
xyz = vertices[:, :3]
w = vertices[:, 3]
colors = get_colors(color_scheme=color_scheme, spacing=spacing, w=w)

In [47]:
plot = k3d.plot(grid_visible=False, menu_visibility=False, background_color=0xFFFFFF, camera_mode="orbit")
points = k3d.points(xyz.astype(np.float32), point_size=0.3, colors=colors)  # The points only support up to float32
plot += points

plot.display()
plot.camera_auto_fit = False
plot.camera = [436660, 3682050, -40] + [436660, 3682050, 80] + [1, 0, 0] # camera position + object position + [up_x, up_y, up_z]


Output()

In [50]:
plot.fetch_screenshot()

In [51]:
with open(save_path, 'wb') as f:
    try:
        out = plot.screenshot.decode('base64')
    except: # Python 3
        from base64 import b64decode
        out = b64decode(plot.screenshot)    
    f.write(out)